In [66]:
import os
import IPython.display as ipd
from scipy.io import wavfile
from collections import Counter

In [58]:
INPUT_DIR = '/d2/caches/tf-speech/train/audio'

In [59]:
def list_wavs(audio_dir):

    labels = [f for f in os.listdir(audio_dir) if f[0] != '_']
    wavs = {}

    for label in labels:
        d = audio_dir + '/' + label
        files = [d + '/' + x for x in os.listdir(d) if x.endswith('.wav')]
        for f in files:
            wavs[f] = label

    return wavs, labels

In [60]:
wavs, labels = list_wavs(INPUT_DIR)
print(len(wavs))

64721


In [74]:
# analyze lengths

sample_rates = Counter()
lengths = Counter()

for w in list(wavs)[:]:
    sr, samples = wavfile.read(w)
    sample_rates[sr] +=1
    l = round(len(samples)/sr, 1)
    lengths[l]+=1

In [75]:
sample_rates

Counter({16000: 64721})

In [76]:
lengths

Counter({0.4: 12,
         0.5: 95,
         0.6: 377,
         0.7: 911,
         0.8: 1315,
         0.9: 2510,
         1.0: 59501})